<a href="https://colab.research.google.com/github/hrithikashukla/Sem6-NLP/blob/master/class%20assignment/Count_Vectoriser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import nltk
import re
nltk.download('words')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split as tts

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv")
df = df[df['Sentiment']!='irrelevant']
df.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...


In [0]:
stop_words = set(stopwords.words('english'))
words = set(nltk.corpus.words.words())

In [0]:
def cleaning(text):
  text = text.lower()
  no_punc = text.translate(text.maketrans('', '', '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~'))
  filtered = re.sub(r'(\s)@\w+', r'\1', no_punc)
  filtered = re.sub(r'(\s)#\w+', r'\1', filtered)
  filtered = filtered.translate(filtered.maketrans('', '', '#@0123456789'))
  tokens = word_tokenize(filtered)
  no_stop_words = [w for w in tokens if not w in stop_words]    
  final = " ".join(w for w in no_stop_words)
  final = " ".join(w for w in nltk.wordpunct_tokenize(final) \
            if w.lower() in words or not w.isalpha())
  
  return final


In [14]:
corpus=df.TweetText.apply(cleaning)
print(corpus)

0                                              get crack
1                                  apple carrier support
2       hilarious video guy duet pretty much love affair
3                            rim made easy switch see ya
4                              reason got twitter thanks
                              ...                       
4537                   modern day autograph like way put
4538                                   ways use business
4539                          log think bout going sleep
4540                                 dumb dont like hush
4541           almost bong rip bowl time space continuum
Name: TweetText, Length: 3424, dtype: object


In [0]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [0]:
X = pd.DataFrame(X.toarray(), columns=(vectorizer.get_feature_names()))

In [17]:
X.columns

Index(['abandoned', 'abbas', 'ability', 'able', 'absolutely', 'abstract',
       'academy', 'accelerate', 'accelerated', 'acceleration',
       ...
       'yet', 'yo', 'you', 'youd', 'young', 'youve', 'yr', 'yummy', 'zero',
       'zoom'],
      dtype='object', length=2796)

In [0]:
X_train, X_test, y_train, y_test = tts(X, df['Sentiment'], test_size=0.3, random_state=42)

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rfc = RandomForestClassifier()

params = {'max_depth': [10, 20, 30, None], 'max_features': ['sqrt'], 'min_samples_leaf': [1, 2, 4], 'min_samples_split': [2, 5, 10], 'n_estimators': range(10,50,10)}
 
grid_rfc = GridSearchCV(estimator = rfc, param_grid = params, cv = 5, n_jobs = 15, verbose = 1)

In [22]:
grid_rfc.fit(X_train,y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=15)]: Using backend LokyBackend with 15 concurrent workers.
[Parallel(n_jobs=15)]: Done  20 tasks      | elapsed:   14.2s
[Parallel(n_jobs=15)]: Done 170 tasks      | elapsed:   46.0s
[Parallel(n_jobs=15)]: Done 420 tasks      | elapsed:  2.3min
[Parallel(n_jobs=15)]: Done 720 out of 720 | elapsed:  6.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [24]:
from sklearn.metrics import classification_report, confusion_matrix
classification_report(grid_rfc.predict(X_test),y_test)

'              precision    recall  f1-score   support\n\n    negative       0.29      0.68      0.41        84\n     neutral       0.95      0.74      0.83       880\n    positive       0.25      0.59      0.35        64\n\n    accuracy                           0.72      1028\n   macro avg       0.50      0.67      0.53      1028\nweighted avg       0.85      0.72      0.77      1028\n'

In [25]:
confusion_matrix(grid_rfc.predict(X_test), y_test)

array([[ 57,  17,  10],
       [127, 647, 106],
       [ 10,  16,  38]])

In [26]:
grid_rfc.best_params_

{'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 40}